In [1]:
from sklearn.cluster import KMeans # KMeans 라이브러리 불러오기
import numpy as np
import pandas as pd
import math
import seaborn as sb # 데이터 시각화를 위한 seaborn
import matplotlib.pyplot as plt # 데이터 시각화를 위한 matplotlib
import scipy as sp
from scipy.spatial import distance
sb.set_palette("Set2")

In [2]:
from google.cloud import bigquery
import pandas as pd

# BigQuery 클라이언트 초기화
client = bigquery.Client(project='jnu-idv-21')

# 쿼리 작성
query = """
SELECT *
FROM `0808.baby_food_tbl2`
"""

# 쿼리 실행 및 결과를 Pandas DataFrame으로 변환
df = client.query(query).to_dataframe()
df

,food_name,ingredient_name,ingredient_index,ingredient_gram,calorie,carbohydrate,protein,fat,fiber,calcium,phosphorus,iron,sodium,potassium,vitamin_A,thiamine,riboflavin,niacin,vitamin_C
0,찹쌀미음,"[쌀, 찹쌀, 백미]","[0, 18, 19]","[-1, -1, 23]",86.000000,18.700001,1.7,0.1,0.1,0.900000,34.700001,0.5,0.700000,43.900002,0.000000,0.0,0.0,0.4,0.0
1,닭가슴살고구마죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 고구마...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 8, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 20]",117.000000,21.400000,6.2,0.4,0.7,9.800000,63.400002,0.5,16.400000,157.199997,4.800000,0.0,0.0,2.7,5.2
2,닭가슴살당근죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 당근,...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 47, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 20]",98.199997,16.900000,6.1,0.4,0.8,12.600000,60.000000,0.5,19.200001,143.800003,252.399994,0.0,0.0,2.7,1.4
3,닭가슴살청경채죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 청경채...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 39]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, 15]",93.500000,15.700000,6.1,0.4,0.7,18.500000,58.299999,0.5,15.600000,105.400002,52.799999,0.0,0.0,2.6,7.4
4,찹쌀컬리플라워당근죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 쌀, 찹쌀, 백미, 컬리플라워,...","[0, 1, 2, 3, 4, 5, 0, 18, 19, 67, 47, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 5, 10, -1, 10]",94.199997,20.700001,2.0,0.1,0.7,9.000000,29.000000,0.5,6.400000,96.000000,125.699997,0.0,0.0,0.5,5.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,닭고기시금치그라탕,"[닭고기, 가슴살, 날것, 시금치, 생것, 노지, 당근, 생것, 양파, 생것, 국내...","[27, 62, 28, 49, 7, 50, 47, 7, 66, 7, 31, 98, ...","[-1, -1, 30, -1, -1, 10, -1, 10, -1, -1, 10, -...",117.599998,4.200000,11.7,6.0,0.8,133.199997,256.600006,0.6,265.100006,223.399994,241.699997,0.1,0.2,3.5,7.9
144,닭죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 분유,...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 63, 64]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 15]",162.000000,23.700001,8.8,3.2,0.2,97.599998,123.099998,1.4,35.200001,144.300003,87.599998,0.1,0.2,3.1,7.0
145,도미살무른밥,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 돔, 감성돔, 배추, 생것, 양...","[0, 1, 2, 3, 4, 5, 83, 84, 40, 7, 73, 7, 52, 53]","[-1, -1, -1, -1, -1, 30, -1, 30, -1, 10, -1, 1...",155.000000,27.500000,9.2,0.7,3.3,89.000000,128.800003,1.4,338.700012,982.599976,15.600000,0.1,0.2,2.2,5.2
146,달걀콩닭고기수프,"[계란, 전란, 생것, 닭고기, 가슴살, 날것, 당근, 생것, 완두콩, 생것, 옥수...","[41, 61, 7, 27, 62, 28, 47, 7, 17, 7, 15, 16, ...","[-1, -1, 20, -1, -1, 20, -1, 10, -1, 10, -1, -...",129.899994,11.200000,10.5,4.6,1.5,84.300003,145.100006,1.6,55.900002,228.100006,221.600006,0.2,0.2,7.5,5.7


### 재료 정보의 평균을 이용하고 재료만을 고려하여 군집화

In [11]:
ingredient_matrix = df.iloc[:,:4]
ingredient_matrix

,food_name,ingredient_name,ingredient_index,ingredient_gram
0,찹쌀미음,"[쌀, 찹쌀, 백미]","[0, 18, 19]","[-1, -1, 23]"
1,닭가슴살고구마죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 고구마...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 8, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 20]"
2,닭가슴살당근죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 당근,...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 47, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 20]"
3,닭가슴살청경채죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 청경채...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 39]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, 15]"
4,찹쌀컬리플라워당근죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 쌀, 찹쌀, 백미, 컬리플라워,...","[0, 1, 2, 3, 4, 5, 0, 18, 19, 67, 47, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 5, 10, -1, 10]"
...,...,...,...,...
143,닭고기시금치그라탕,"[닭고기, 가슴살, 날것, 시금치, 생것, 노지, 당근, 생것, 양파, 생것, 국내...","[27, 62, 28, 49, 7, 50, 47, 7, 66, 7, 31, 98, ...","[-1, -1, 30, -1, -1, 10, -1, 10, -1, -1, 10, -..."
144,닭죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 분유,...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 63, 64]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 15]"
145,도미살무른밥,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 돔, 감성돔, 배추, 생것, 양...","[0, 1, 2, 3, 4, 5, 83, 84, 40, 7, 73, 7, 52, 53]","[-1, -1, -1, -1, -1, 30, -1, 30, -1, 10, -1, 1..."
146,달걀콩닭고기수프,"[계란, 전란, 생것, 닭고기, 가슴살, 날것, 당근, 생것, 완두콩, 생것, 옥수...","[41, 61, 7, 27, 62, 28, 47, 7, 17, 7, 15, 16, ...","[-1, -1, 20, -1, -1, 20, -1, 10, -1, 10, -1, -..."


In [12]:
# 데이터 전처리

# 평균 값을 사용하여 데이터를 변환
ingredient_matrix['ingredient_index_avg'] = ingredient_matrix['ingredient_index'].apply(lambda x: sum(x) / len(x))
ingredient_matrix['ingredient_gram_avg'] = ingredient_matrix['ingredient_gram'].apply(lambda x: sum(x) / len(x))

# 데이터 스케일링
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_data = scaler.fit_transform(ingredient_matrix[['ingredient_index_avg', 'ingredient_gram_avg']])

# KMeans 군집화 모델 생성
num_clusters = 3  # 군집 개수 설정
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(scaled_data)

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=3, random_state=42)

In [14]:
# 군집 결과를 데이터프레임에 추가
ingredient_matrix['cluster'] = kmeans.labels_
ingredient_matrix

,food_name,ingredient_name,ingredient_index,ingredient_gram,ingredient_index_avg,ingredient_gram_avg,cluster
0,찹쌀미음,"[쌀, 찹쌀, 백미]","[0, 18, 19]","[-1, -1, 23]",12.333333,7.000000,1
1,닭가슴살고구마죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 고구마...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 8, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 20]",13.363636,4.727273,1
2,닭가슴살당근죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 당근,...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 47, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 20]",16.909091,4.727273,1
3,닭가슴살청경채죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 청경채...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 39]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, 15]",17.100000,4.800000,1
4,찹쌀컬리플라워당근죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 쌀, 찹쌀, 백미, 컬리플라워,...","[0, 1, 2, 3, 4, 5, 0, 18, 19, 67, 47, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 5, 10, -1, 10]",14.416667,3.083333,1
...,...,...,...,...,...,...,...
143,닭고기시금치그라탕,"[닭고기, 가슴살, 날것, 시금치, 생것, 노지, 당근, 생것, 양파, 생것, 국내...","[27, 62, 28, 49, 7, 50, 47, 7, 66, 7, 31, 98, ...","[-1, -1, 30, -1, -1, 10, -1, 10, -1, -1, 10, -...",51.187500,5.625000,0
144,닭죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 분유,...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 63, 64]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 15]",23.545455,4.272727,1
145,도미살무른밥,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 돔, 감성돔, 배추, 생것, 양...","[0, 1, 2, 3, 4, 5, 83, 84, 40, 7, 73, 7, 52, 53]","[-1, -1, -1, -1, -1, 30, -1, 30, -1, 10, -1, 1...",29.571429,5.785714,0
146,달걀콩닭고기수프,"[계란, 전란, 생것, 닭고기, 가슴살, 날것, 당근, 생것, 완두콩, 생것, 옥수...","[41, 61, 7, 27, 62, 28, 47, 7, 17, 7, 15, 16, ...","[-1, -1, 20, -1, -1, 20, -1, 10, -1, 10, -1, -...",31.266667,4.733333,0


In [21]:
# 군집화 확인 : cluster0

cluster0 = 0
cluster0_num = (df['cluster']==0).sum()
print(f'cluster0에 들어있는 음식 종류 수:{cluster0_num}')

cluster0에 들어있는 음식 종류 수:31


In [22]:
# 군집화 확인 : cluster1

cluster1 = 1
cluster1_num = (df['cluster']==1).sum()
print(f'cluster1에 들어있는 음식 종류 수:{cluster1_num}')

cluster1에 들어있는 음식 종류 수:112


In [24]:
# 군집화 확인 : cluster2

cluster2 = 2
cluster2_num = (df['cluster']==2).sum()
print(f'cluster2에 들어있는 음식 종류 수:{cluster2_num}')
# 망했다

cluster2에 들어있는 음식 종류 수:5


### 재료 정보의 평균을 이용하고 그 외 feature 포함하여 군집화

In [34]:
# 데이터 전처리

# 평균 값을 사용하여 데이터를 변환
df['ingredient_index_avg'] = df['ingredient_index'].apply(lambda x: sum(x) / len(x))
df['ingredient_gram_avg'] = df['ingredient_gram'].apply(lambda x: sum(x) / len(x))

# 데이터 스케일링
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df[['ingredient_index_avg', 'ingredient_gram_avg']])

scaled_data = pd.DataFrame(scaled_data)
scaled_data

# 전처리된 데이터프레임 합치기
merged_df = pd.concat([df, scaled_data],axis=1)
merged_df

,food_name,ingredient_name,ingredient_index,ingredient_gram,calorie,carbohydrate,protein,fat,fiber,calcium,...,vitamin_A,thiamine,riboflavin,niacin,vitamin_C,ingredient_index_avg,ingredient_gram_avg,cluster,0,1
0,찹쌀미음,"[쌀, 찹쌀, 백미]","[0, 18, 19]","[-1, -1, 23]",86.000000,18.700001,1.7,0.1,0.1,0.900000,...,0.000000,0.0,0.0,0.4,0.0,12.333333,7.000000,1,-0.482140,0.193967
1,닭가슴살고구마죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 고구마...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 8, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 20]",117.000000,21.400000,6.2,0.4,0.7,9.800000,...,4.800000,0.0,0.0,2.7,5.2,13.363636,4.727273,1,-0.401581,-0.157370
2,닭가슴살당근죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 당근,...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 47, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 20]",98.199997,16.900000,6.1,0.4,0.8,12.600000,...,252.399994,0.0,0.0,2.7,1.4,16.909091,4.727273,1,-0.124364,-0.157370
3,닭가슴살청경채죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 청경채...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 39]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, 15]",93.500000,15.700000,6.1,0.4,0.7,18.500000,...,52.799999,0.0,0.0,2.6,7.4,17.100000,4.800000,1,-0.109437,-0.146127
4,찹쌀컬리플라워당근죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 쌀, 찹쌀, 백미, 컬리플라워,...","[0, 1, 2, 3, 4, 5, 0, 18, 19, 67, 47, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 5, 10, -1, 10]",94.199997,20.700001,2.0,0.1,0.7,9.000000,...,125.699997,0.0,0.0,0.5,5.4,14.416667,3.083333,1,-0.319245,-0.411503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,닭고기시금치그라탕,"[닭고기, 가슴살, 날것, 시금치, 생것, 노지, 당근, 생것, 양파, 생것, 국내...","[27, 62, 28, 49, 7, 50, 47, 7, 66, 7, 31, 98, ...","[-1, -1, 30, -1, -1, 10, -1, 10, -1, -1, 10, -...",117.599998,4.200000,11.7,6.0,0.8,133.199997,...,241.699997,0.1,0.2,3.5,7.9,51.187500,5.625000,0,2.555850,-0.018592
144,닭죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 분유,...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 63, 64]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 15]",162.000000,23.700001,8.8,3.2,0.2,97.599998,...,87.599998,0.1,0.2,3.1,7.0,23.545455,4.272727,1,0.394530,-0.227637
145,도미살무른밥,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 돔, 감성돔, 배추, 생것, 양...","[0, 1, 2, 3, 4, 5, 83, 84, 40, 7, 73, 7, 52, 53]","[-1, -1, -1, -1, -1, 30, -1, 30, -1, 10, -1, 1...",155.000000,27.500000,9.2,0.7,3.3,89.000000,...,15.600000,0.1,0.2,2.2,5.2,29.571429,5.785714,0,0.865699,0.006253
146,달걀콩닭고기수프,"[계란, 전란, 생것, 닭고기, 가슴살, 날것, 당근, 생것, 완두콩, 생것, 옥수...","[41, 61, 7, 27, 62, 28, 47, 7, 17, 7, 15, 16, ...","[-1, -1, 20, -1, -1, 20, -1, 10, -1, 10, -1, -...",129.899994,11.200000,10.5,4.6,1.5,84.300003,...,221.600006,0.2,0.2,7.5,5.7,31.266667,4.733333,0,0.998249,-0.156433


In [50]:
all_matrix = merged_df.iloc[:,4:]

# feature에 int,str 동시 존재 x
# 정수형 열 -> 문자열 열
column = {0:'scaled_0',1:'scaled_1'}
all_matrix.rename(columns=column,inplace=True)
all_matrix

,calorie,carbohydrate,protein,fat,fiber,calcium,phosphorus,iron,sodium,potassium,vitamin_A,thiamine,riboflavin,niacin,vitamin_C,ingredient_index_avg,ingredient_gram_avg,cluster,scaled_0,scaled_1
0,86.000000,18.700001,1.7,0.1,0.1,0.900000,34.700001,0.5,0.700000,43.900002,0.000000,0.0,0.0,0.4,0.0,12.333333,7.000000,1,-0.482140,0.193967
1,117.000000,21.400000,6.2,0.4,0.7,9.800000,63.400002,0.5,16.400000,157.199997,4.800000,0.0,0.0,2.7,5.2,13.363636,4.727273,1,-0.401581,-0.157370
2,98.199997,16.900000,6.1,0.4,0.8,12.600000,60.000000,0.5,19.200001,143.800003,252.399994,0.0,0.0,2.7,1.4,16.909091,4.727273,1,-0.124364,-0.157370
3,93.500000,15.700000,6.1,0.4,0.7,18.500000,58.299999,0.5,15.600000,105.400002,52.799999,0.0,0.0,2.6,7.4,17.100000,4.800000,1,-0.109437,-0.146127
4,94.199997,20.700001,2.0,0.1,0.7,9.000000,29.000000,0.5,6.400000,96.000000,125.699997,0.0,0.0,0.5,5.4,14.416667,3.083333,1,-0.319245,-0.411503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,117.599998,4.200000,11.7,6.0,0.8,133.199997,256.600006,0.6,265.100006,223.399994,241.699997,0.1,0.2,3.5,7.9,51.187500,5.625000,0,2.555850,-0.018592
144,162.000000,23.700001,8.8,3.2,0.2,97.599998,123.099998,1.4,35.200001,144.300003,87.599998,0.1,0.2,3.1,7.0,23.545455,4.272727,1,0.394530,-0.227637
145,155.000000,27.500000,9.2,0.7,3.3,89.000000,128.800003,1.4,338.700012,982.599976,15.600000,0.1,0.2,2.2,5.2,29.571429,5.785714,0,0.865699,0.006253
146,129.899994,11.200000,10.5,4.6,1.5,84.300003,145.100006,1.6,55.900002,228.100006,221.600006,0.2,0.2,7.5,5.7,31.266667,4.733333,0,0.998249,-0.156433


In [51]:
# KMeans 군집화 모델 생성

num_clusters = 3  # 군집 개수 설정
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(all_matrix)

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=3, random_state=42)

In [52]:
# 군집 결과를 데이터프레임에 추가
df['cluster'] = kmeans.labels_
df

,food_name,ingredient_name,ingredient_index,ingredient_gram,calorie,carbohydrate,protein,fat,fiber,calcium,...,sodium,potassium,vitamin_A,thiamine,riboflavin,niacin,vitamin_C,ingredient_index_avg,ingredient_gram_avg,cluster
0,찹쌀미음,"[쌀, 찹쌀, 백미]","[0, 18, 19]","[-1, -1, 23]",86.000000,18.700001,1.7,0.1,0.1,0.900000,...,0.700000,43.900002,0.000000,0.0,0.0,0.4,0.0,12.333333,7.000000,0
1,닭가슴살고구마죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 고구마...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 8, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 20]",117.000000,21.400000,6.2,0.4,0.7,9.800000,...,16.400000,157.199997,4.800000,0.0,0.0,2.7,5.2,13.363636,4.727273,0
2,닭가슴살당근죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 당근,...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 47, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 20]",98.199997,16.900000,6.1,0.4,0.8,12.600000,...,19.200001,143.800003,252.399994,0.0,0.0,2.7,1.4,16.909091,4.727273,1
3,닭가슴살청경채죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 청경채...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 39]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, 15]",93.500000,15.700000,6.1,0.4,0.7,18.500000,...,15.600000,105.400002,52.799999,0.0,0.0,2.6,7.4,17.100000,4.800000,0
4,찹쌀컬리플라워당근죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 쌀, 찹쌀, 백미, 컬리플라워,...","[0, 1, 2, 3, 4, 5, 0, 18, 19, 67, 47, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 5, 10, -1, 10]",94.199997,20.700001,2.0,0.1,0.7,9.000000,...,6.400000,96.000000,125.699997,0.0,0.0,0.5,5.4,14.416667,3.083333,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,닭고기시금치그라탕,"[닭고기, 가슴살, 날것, 시금치, 생것, 노지, 당근, 생것, 양파, 생것, 국내...","[27, 62, 28, 49, 7, 50, 47, 7, 66, 7, 31, 98, ...","[-1, -1, 30, -1, -1, 10, -1, 10, -1, -1, 10, -...",117.599998,4.200000,11.7,6.0,0.8,133.199997,...,265.100006,223.399994,241.699997,0.1,0.2,3.5,7.9,51.187500,5.625000,1
144,닭죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 분유,...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 63, 64]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 15]",162.000000,23.700001,8.8,3.2,0.2,97.599998,...,35.200001,144.300003,87.599998,0.1,0.2,3.1,7.0,23.545455,4.272727,0
145,도미살무른밥,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 돔, 감성돔, 배추, 생것, 양...","[0, 1, 2, 3, 4, 5, 83, 84, 40, 7, 73, 7, 52, 53]","[-1, -1, -1, -1, -1, 30, -1, 30, -1, 10, -1, 1...",155.000000,27.500000,9.2,0.7,3.3,89.000000,...,338.700012,982.599976,15.600000,0.1,0.2,2.2,5.2,29.571429,5.785714,2
146,달걀콩닭고기수프,"[계란, 전란, 생것, 닭고기, 가슴살, 날것, 당근, 생것, 완두콩, 생것, 옥수...","[41, 61, 7, 27, 62, 28, 47, 7, 17, 7, 15, 16, ...","[-1, -1, 20, -1, -1, 20, -1, 10, -1, 10, -1, -...",129.899994,11.200000,10.5,4.6,1.5,84.300003,...,55.900002,228.100006,221.600006,0.2,0.2,7.5,5.7,31.266667,4.733333,1


In [53]:
# 군집화 확인 : cluster0

cluster0 = 0
cluster0_num = (df['cluster']==0).sum()
print(f'cluster0에 들어있는 음식 종류 수:{cluster0_num}')

cluster0에 들어있는 음식 종류 수:116


In [54]:
# 군집화 확인 : cluster1

cluster1 = 1
cluster1_num = (df['cluster']==1).sum()
print(f'cluster1에 들어있는 음식 종류 수:{cluster1_num}')

cluster1에 들어있는 음식 종류 수:29


In [55]:
# 군집화 확인 : cluster2

cluster2 = 2
cluster2_num = (df['cluster']==2).sum()
print(f'cluster2에 들어있는 음식 종류 수:{cluster2_num}')
# 망했다

cluster2에 들어있는 음식 종류 수:3


### 전처리 지표 제외하고 다시 해보기

In [56]:
all_matrix

,calorie,carbohydrate,protein,fat,fiber,calcium,phosphorus,iron,sodium,potassium,vitamin_A,thiamine,riboflavin,niacin,vitamin_C,ingredient_index_avg,ingredient_gram_avg,cluster,scaled_0,scaled_1
0,86.000000,18.700001,1.7,0.1,0.1,0.900000,34.700001,0.5,0.700000,43.900002,0.000000,0.0,0.0,0.4,0.0,12.333333,7.000000,1,-0.482140,0.193967
1,117.000000,21.400000,6.2,0.4,0.7,9.800000,63.400002,0.5,16.400000,157.199997,4.800000,0.0,0.0,2.7,5.2,13.363636,4.727273,1,-0.401581,-0.157370
2,98.199997,16.900000,6.1,0.4,0.8,12.600000,60.000000,0.5,19.200001,143.800003,252.399994,0.0,0.0,2.7,1.4,16.909091,4.727273,1,-0.124364,-0.157370
3,93.500000,15.700000,6.1,0.4,0.7,18.500000,58.299999,0.5,15.600000,105.400002,52.799999,0.0,0.0,2.6,7.4,17.100000,4.800000,1,-0.109437,-0.146127
4,94.199997,20.700001,2.0,0.1,0.7,9.000000,29.000000,0.5,6.400000,96.000000,125.699997,0.0,0.0,0.5,5.4,14.416667,3.083333,1,-0.319245,-0.411503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,117.599998,4.200000,11.7,6.0,0.8,133.199997,256.600006,0.6,265.100006,223.399994,241.699997,0.1,0.2,3.5,7.9,51.187500,5.625000,0,2.555850,-0.018592
144,162.000000,23.700001,8.8,3.2,0.2,97.599998,123.099998,1.4,35.200001,144.300003,87.599998,0.1,0.2,3.1,7.0,23.545455,4.272727,1,0.394530,-0.227637
145,155.000000,27.500000,9.2,0.7,3.3,89.000000,128.800003,1.4,338.700012,982.599976,15.600000,0.1,0.2,2.2,5.2,29.571429,5.785714,0,0.865699,0.006253
146,129.899994,11.200000,10.5,4.6,1.5,84.300003,145.100006,1.6,55.900002,228.100006,221.600006,0.2,0.2,7.5,5.7,31.266667,4.733333,0,0.998249,-0.156433


In [60]:
fix_matrix = all_matrix.iloc[:,:-3]
fix_matrix

,calorie,carbohydrate,protein,fat,fiber,calcium,phosphorus,iron,sodium,potassium,vitamin_A,thiamine,riboflavin,niacin,vitamin_C,ingredient_index_avg,ingredient_gram_avg
0,86.000000,18.700001,1.7,0.1,0.1,0.900000,34.700001,0.5,0.700000,43.900002,0.000000,0.0,0.0,0.4,0.0,12.333333,7.000000
1,117.000000,21.400000,6.2,0.4,0.7,9.800000,63.400002,0.5,16.400000,157.199997,4.800000,0.0,0.0,2.7,5.2,13.363636,4.727273
2,98.199997,16.900000,6.1,0.4,0.8,12.600000,60.000000,0.5,19.200001,143.800003,252.399994,0.0,0.0,2.7,1.4,16.909091,4.727273
3,93.500000,15.700000,6.1,0.4,0.7,18.500000,58.299999,0.5,15.600000,105.400002,52.799999,0.0,0.0,2.6,7.4,17.100000,4.800000
4,94.199997,20.700001,2.0,0.1,0.7,9.000000,29.000000,0.5,6.400000,96.000000,125.699997,0.0,0.0,0.5,5.4,14.416667,3.083333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,117.599998,4.200000,11.7,6.0,0.8,133.199997,256.600006,0.6,265.100006,223.399994,241.699997,0.1,0.2,3.5,7.9,51.187500,5.625000
144,162.000000,23.700001,8.8,3.2,0.2,97.599998,123.099998,1.4,35.200001,144.300003,87.599998,0.1,0.2,3.1,7.0,23.545455,4.272727
145,155.000000,27.500000,9.2,0.7,3.3,89.000000,128.800003,1.4,338.700012,982.599976,15.600000,0.1,0.2,2.2,5.2,29.571429,5.785714
146,129.899994,11.200000,10.5,4.6,1.5,84.300003,145.100006,1.6,55.900002,228.100006,221.600006,0.2,0.2,7.5,5.7,31.266667,4.733333


In [61]:
# KMeans 군집화 모델 생성
num_clusters = 3  # 군집 개수 설정
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(fix_matrix)

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=3, random_state=42)

In [62]:
# 군집 결과를 데이터프레임에 추가
df['cluster'] = kmeans.labels_
df

,food_name,ingredient_name,ingredient_index,ingredient_gram,calorie,carbohydrate,protein,fat,fiber,calcium,...,sodium,potassium,vitamin_A,thiamine,riboflavin,niacin,vitamin_C,ingredient_index_avg,ingredient_gram_avg,cluster
0,찹쌀미음,"[쌀, 찹쌀, 백미]","[0, 18, 19]","[-1, -1, 23]",86.000000,18.700001,1.7,0.1,0.1,0.900000,...,0.700000,43.900002,0.000000,0.0,0.0,0.4,0.0,12.333333,7.000000,0
1,닭가슴살고구마죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 고구마...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 8, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 20]",117.000000,21.400000,6.2,0.4,0.7,9.800000,...,16.400000,157.199997,4.800000,0.0,0.0,2.7,5.2,13.363636,4.727273,0
2,닭가슴살당근죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 당근,...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 47, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 20]",98.199997,16.900000,6.1,0.4,0.8,12.600000,...,19.200001,143.800003,252.399994,0.0,0.0,2.7,1.4,16.909091,4.727273,1
3,닭가슴살청경채죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 청경채...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 39]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, 15]",93.500000,15.700000,6.1,0.4,0.7,18.500000,...,15.600000,105.400002,52.799999,0.0,0.0,2.6,7.4,17.100000,4.800000,0
4,찹쌀컬리플라워당근죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 쌀, 찹쌀, 백미, 컬리플라워,...","[0, 1, 2, 3, 4, 5, 0, 18, 19, 67, 47, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 5, 10, -1, 10]",94.199997,20.700001,2.0,0.1,0.7,9.000000,...,6.400000,96.000000,125.699997,0.0,0.0,0.5,5.4,14.416667,3.083333,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,닭고기시금치그라탕,"[닭고기, 가슴살, 날것, 시금치, 생것, 노지, 당근, 생것, 양파, 생것, 국내...","[27, 62, 28, 49, 7, 50, 47, 7, 66, 7, 31, 98, ...","[-1, -1, 30, -1, -1, 10, -1, 10, -1, -1, 10, -...",117.599998,4.200000,11.7,6.0,0.8,133.199997,...,265.100006,223.399994,241.699997,0.1,0.2,3.5,7.9,51.187500,5.625000,1
144,닭죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 분유,...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 63, 64]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 15]",162.000000,23.700001,8.8,3.2,0.2,97.599998,...,35.200001,144.300003,87.599998,0.1,0.2,3.1,7.0,23.545455,4.272727,0
145,도미살무른밥,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 돔, 감성돔, 배추, 생것, 양...","[0, 1, 2, 3, 4, 5, 83, 84, 40, 7, 73, 7, 52, 53]","[-1, -1, -1, -1, -1, 30, -1, 30, -1, 10, -1, 1...",155.000000,27.500000,9.2,0.7,3.3,89.000000,...,338.700012,982.599976,15.600000,0.1,0.2,2.2,5.2,29.571429,5.785714,2
146,달걀콩닭고기수프,"[계란, 전란, 생것, 닭고기, 가슴살, 날것, 당근, 생것, 완두콩, 생것, 옥수...","[41, 61, 7, 27, 62, 28, 47, 7, 17, 7, 15, 16, ...","[-1, -1, 20, -1, -1, 20, -1, 10, -1, 10, -1, -...",129.899994,11.200000,10.5,4.6,1.5,84.300003,...,55.900002,228.100006,221.600006,0.2,0.2,7.5,5.7,31.266667,4.733333,1


In [63]:
# 군집화 확인 : cluster0

cluster0 = 0
cluster0_num = (df['cluster']==0).sum()
print(f'cluster0에 들어있는 음식 종류 수:{cluster0_num}')

cluster0에 들어있는 음식 종류 수:116


In [64]:
# 군집화 확인 : cluster1

cluster1 = 1
cluster1_num = (df['cluster']==1).sum()
print(f'cluster1에 들어있는 음식 종류 수:{cluster1_num}')

cluster1에 들어있는 음식 종류 수:29


In [65]:
# 군집화 확인 : cluster2

cluster2 = 2
cluster2_num = (df['cluster']==2).sum()
print(f'cluster2에 들어있는 음식 종류 수:{cluster2_num}')
# 망했다

cluster2에 들어있는 음식 종류 수:3
